In [1]:
import sqlite3
import pandas as pd
import numpy as np
import tensorflow as tf
from pickle import dump

In [2]:
tf.__version__

'2.7.0'

# Connect to our DB

In [3]:
con = sqlite3.connect('riot_matches.db')

In [4]:
df_matches =  pd.read_sql_query("SELECT * FROM match", con)
con.close()

In [5]:
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13374 entries, 0 to 13373
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            13374 non-null  int64 
 1   game_id       13374 non-null  object
 2   blue_dragons  13374 non-null  int64 
 3   blue_heralds  13374 non-null  int64 
 4   blue_barons   13374 non-null  int64 
 5   blue_turrets  13374 non-null  int64 
 6   blue_kills    13374 non-null  int64 
 7   blue_gold_20  13374 non-null  int64 
 8   red_dragons   13374 non-null  int64 
 9   red_heralds   13374 non-null  int64 
 10  red_barons    13374 non-null  int64 
 11  red_turrets   13374 non-null  int64 
 12  red_kills     13374 non-null  int64 
 13  red_gold_20   13374 non-null  int64 
 14  winner        13374 non-null  object
dtypes: int64(13), object(2)
memory usage: 1.5+ MB


In [6]:
df_matches.sample(5)

,id,game_id,blue_dragons,blue_heralds,blue_barons,blue_turrets,blue_kills,blue_gold_20,red_dragons,red_heralds,red_barons,red_turrets,red_kills,red_gold_20,winner
6209,6210,NA1_4187557861,1,1,0,6,30,48432,1,1,0,5,14,39490,Team 1
1352,1353,NA1_4182707217,2,2,0,13,26,46578,0,0,0,0,14,34322,Team 1
6317,6318,NA1_4191366040,1,1,0,2,18,38100,2,1,0,6,32,48711,Team 2
13006,13007,NA1_4190365913,1,0,1,1,21,40187,2,2,0,3,18,40531,Team 1
12548,12549,NA1_4189353583,1,0,0,1,18,38699,2,2,1,7,36,51433,Team 2


# Chosen Parameters

Compared to the gold-only model, this model will utilize more parameters that influence momentum and game state. 
<br>The chosen parameters are:
<br>-Kills 
<br>-Deaths
<br>-Assists
<br>-Dragons
<br>-Heralds
<br>-Gold Difference
<br>-Objectives(Turrets/Inhibitors)
<br>These parameters would be calculated in each team's perspective, and then used in our neural network as a 1D array

# Clean our database entries

In [7]:
df_matches = df_matches.drop_duplicates(subset=['game_id'])
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5783 entries, 0 to 13373
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            5783 non-null   int64 
 1   game_id       5783 non-null   object
 2   blue_dragons  5783 non-null   int64 
 3   blue_heralds  5783 non-null   int64 
 4   blue_barons   5783 non-null   int64 
 5   blue_turrets  5783 non-null   int64 
 6   blue_kills    5783 non-null   int64 
 7   blue_gold_20  5783 non-null   int64 
 8   red_dragons   5783 non-null   int64 
 9   red_heralds   5783 non-null   int64 
 10  red_barons    5783 non-null   int64 
 11  red_turrets   5783 non-null   int64 
 12  red_kills     5783 non-null   int64 
 13  red_gold_20   5783 non-null   int64 
 14  winner        5783 non-null   object
dtypes: int64(13), object(2)
memory usage: 722.9+ KB


In [8]:
df_matches.reset_index(inplace=True, drop=True)
df_matches['winner'] = df_matches.apply(lambda row:
     1 if row['winner'] == 'Team 1'
     else 0,
     axis=1
)
df_matches

,id,game_id,blue_dragons,blue_heralds,blue_barons,blue_turrets,blue_kills,blue_gold_20,red_dragons,red_heralds,red_barons,red_turrets,red_kills,red_gold_20,winner
0,1,NA1_4191213709,1,1,0,4,33,46758,2,0,0,3,27,43584,0
1,2,NA1_4191210746,2,1,0,5,25,43593,1,1,0,2,8,34308,1
2,3,NA1_4191135959,3,2,0,12,28,46983,0,0,0,0,12,33526,1
3,4,NA1_4191130723,0,0,1,1,25,41676,1,1,0,3,22,40890,0
4,5,NA1_4189716443,1,2,0,8,27,48079,1,0,0,2,14,38229,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5778,13370,NA1_4190634273,3,1,1,5,28,45054,0,1,0,1,15,36413,1
5779,13371,NA1_4190605222,0,2,0,2,17,39224,2,0,0,4,11,38690,0
5780,13372,NA1_4184950229,0,0,0,2,18,38409,3,2,0,3,15,39227,0
5781,13373,NA1_4184835767,0,0,0,1,20,40145,3,2,0,3,36,47098,0


In [9]:
df_features = df_matches.iloc[:, 2:14].values
df_features.shape

(5783, 12)

In [10]:
df_label = df_matches.iloc[:, -1:].values
df_label

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]], dtype=int64)

### Split Data to training/testing features and label

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_features, df_label, test_size=0.2, random_state=0)

In [12]:
print(x_train[0], y_train[0])

[    3     1     0     4    11 38727     0     1     0     3    11 36581] [1]


### Apply Standardization to data (helps the model with assigning weights and bias)

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler().fit(x_train)
x_train = sc.transform(x_train)
x_test = sc.transform(x_test)
dump(sc, open('scaler.pkl', 'wb')) # Save our mean and standard deviation for prediction

In [14]:
x_train[0]

array([ 1.82661646,  0.09737731, -0.26427777,  0.50996733, -1.06670722,
       -0.23249149, -1.47301635,  0.08439657, -0.27851056,  0.04976615,
       -1.09563746, -0.82078816])

In [15]:
(x_test[0].shape)

(12,)

# Create our sequential model

In [16]:
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=8, activation="relu"))
nn.add(tf.keras.layers.Dense(units=4, activation="relu"))
# nn.add(tf.keras.layers.Dense(units=2, activation="relu"))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
nn.fit(x_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
145/145 [==============================] - 1s 3ms/step - loss: 0.5895 - accuracy: 0.6922
Epoch 2/100
145/145 [==============================] - 0s 3ms/step - loss: 0.4520 - accuracy: 0.7892
Epoch 3/100
145/145 [==============================] - 0s 3ms/step - loss: 0.4189 - accuracy: 0.8080
Epoch 4/100
145/145 [==============================] - 0s 3ms/step - loss: 0.4088 - accuracy: 0.8115
Epoch 5/100
145/145 [==============================] - 0s 3ms/step - loss: 0.4029 - accuracy: 0.8150
Epoch 6/100
145/145 [==============================] - 0s 3ms/step - loss: 0.3996 - accuracy: 0.8160
Epoch 7/100
145/145 [==============================] - 0s 3ms/step - loss: 0.3969 - accuracy: 0.8171
Epoch 8/100
145/145 [==============================] - 0s 3ms/step - loss: 0.3953 - accuracy: 0.8167
Epoch 9/100
145/145 [==============================] - 0s 3ms/step - loss: 0.3937 - accuracy: 0.8186
Epoch 10/100
145/145 [==============================] - 0s 3ms/step - loss: 0.3923 - accura

In [17]:
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 104       
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 145
Trainable params: 145
Non-trainable params: 0
_________________________________________________________________


In [18]:
results = nn.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

10/10 [==============================] - 0s 2ms/step - loss: 0.3912 - accuracy: 0.8133
test loss, test acc: [0.39115676283836365, 0.8133102655410767]


In [19]:
# nn.save("model_w_params")

In [20]:
# model = tf.keras.models.load_model('model_w_paramsfeb1')

In [21]:
print(x_test[0], y_test[0])
print(nn.predict(np.array([x_test[0]])), y_test[0])

[-1.25542406 -1.2007997  -0.26427777  0.50996733 -1.21771825 -0.45263979
  1.57446738  1.40337818 -0.27851056  0.52739728 -0.33502474  0.01255354] [0]
[[0.15510498]] [0]


In [22]:
print(x_test[1], y_test[1])
print(nn.predict(np.array([x_test[1]])), y_test[1])

[-0.22807722 -1.2007997   3.78389757 -0.40850994 -0.46266308  0.10933128
 -0.45718844  0.08439657 -0.27851056 -0.42786498 -0.63926983 -0.7617524 ] [1]
[[0.905683]] [1]
